# Naive Bayes

V tomto notebooku se budeme zabývat využitím klasifikace pomocí Naivního Bayese. Speciálně se budeme soustředit na klasifikaci textů.
 
Základem pro tento dokument je tutorial ze scikit-learn zaměřený na analýzu textů [zde](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [1]:
import pandas as pd
import numpy as np
from scipy.misc import logsumexp
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.)

## Načtení dat

Využijeme data ze zdroje [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), který obsahuje různě kategorizované texty z internetových diskusí.

Pro jednoduchost se zaměříme pouze na dvě kategorie - hokej a auta.

In [2]:
categories = ['rec.sport.hockey', 'rec.autos']
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=1)
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=1)

In [13]:
# Prozkoumání testovacích dat
print('Kategorie:', test_data.target_names)
print('Train data length:', len(train_data.data))

#print(train_data.data[0])
print('Kategorie prvního dokumentu:',train_data.target_names[train_data.target[0]])

Kategorie: ['rec.autos', 'rec.sport.hockey']
Train data length: 1194
From: rmt6r@faraday.clas.Virginia.EDU (Roy Matthew Thigpen)
Subject: Re: Ad said Nissan Altima best seller?
Organization: University of Virginia
Lines: 35

boyle@cactus.org  writes:
> In article <1qv7mn$dql@menudo.uh.edu> thang@harebell.egr.uh.edu (Chin-Heng  Thang) writes:
> >	Recently, I saw an ad for the altima which says that it is the  
> >best seller for the past 6 months, is that true? 
> >
> 
> I too was puzzled by this obvious untruth. What I think is going on is that
> Nissan claims that the Altima is "the best selling new car namelplate in
> the US" (I think I have this near verbatim). Lee Iaccoca's statistics
> dept. would have been proud of that sentence. What they mean, I think, is
> that of all "totally new models", i.e. cars never sold before in any
> form, the Altima is the best seller, thereby eliminating Accord, Taurus
> etc. 
 THis is from the same people who make the claim that our minivan is outs

## Transformace do bag-of-words reprezentace

Použijeme k tomu CountVectorizer ze scikit-learn.

In [21]:
# Nejprve načteme slovník
with open('vocabulary.txt','r') as f: #musi podporovat __enter__() a __exit__()
    vocab=f.read().splitlines()
print(len(vocab))

61188


In [22]:
count_vect = CountVectorizer(vocabulary = vocab)
X_train_counts = count_vect.fit_transform(train_data.data) #
print('Bag of words shape', X_train_counts.shape)
print('Bag of words type', type(X_train_counts))

Bag of words shape (1194, 61188)
Bag of words type <class 'scipy.sparse.csr.csr_matrix'>


Výstupem je scipy.sparse matrix.

In [6]:
# zobrazení prvního řádku - tj.příznaků prvního dokumentu
X_train_counts[0,:20].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 2, 0, 0, 0, 0]])

We can also extract the vocabulary...

In [7]:
# several words from the dictionary together with their indices in the dictionary
print(type(count_vect.vocabulary_))
print(len(count_vect.vocabulary_))
print(len(vocab))

print({vocab[i]:X_train_counts[0,i] for i in range(20)})

<class 'dict'>
61188
61188
{'archive': 0, 'name': 0, 'atheism': 0, 'resources': 0, 'alt': 0, 'last': 0, 'modified': 0, 'december': 0, 'version': 0, 'atheist': 0, 'addresses': 0, 'of': 6, 'organizations': 0, 'usa': 0, 'freedom': 0, 'from': 2, 'religion': 0, 'foundation': 0, 'darwin': 0, 'fish': 0}


### TASK 1 - aplikujte nejprve jednoduchý model - Bernoulli Naive Bayes

* Reprezentujte dokument pomocí indikátorů výskytů slov ze slovníku vocab
* Natrénujte Naivního Bayese s Bernoulliho rozdělením příznaků
* Otestujte kvalitu predikce na ručně určených dokumentech
* Odhadněte přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [24]:
train_data.

array([0, 1, 0, ..., 0, 0, 0])

In [34]:
# Vlastní dokumenty pro testování
docs_new = ["Lets play hockey.", "I don't like their seats"]

count_v = CountVectorizer(vocabulary = vocab, binary=True)
X_train_counts = count_v.fit_transform(train_data.data)
x_target_data = count_v.fit_transform(test_data.data)
clf = BernoulliNB()
clf.fit(X_train_counts, train_data.target)

predicted = clf.predict(x_target_data)
probabs   = clf.predict_proba(x_target_data)
trues= 0
falses= 0
print(np.mean(predicted == test_data.target))
print(clf.score(x_target_data,test_data.target))
# alternatives to play with
# docs_new = ["Lets play hockey.", "I don't like their game"]

0.9911949685534591
0.9911949685534591


In [9]:
# Your code here


### TASK 2 - aplikujte složitější model - Multinomial Naive Bayes

* Reprezentujte dokument pomocí počtů výskytů slov ze slovníku vocab - tj. bag-of-words reprezentace
* Natrénujte multinomického naivního Bayese
* Otestujte kvalitu predikce na ručně určených dokumentech
* Odhadněte přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [40]:
# Your code here
#docs_new = ["Lets play hockey.", "I don't like their game"]
docs_new = ["Lets play hockey.", "I don't like their seats"]

count_v = CountVectorizer(vocabulary = vocab, binary=False)
X_train_counts = count_v.fit_transform(train_data.data)
x_target_data = count_v.transform(test_data.data)
example_data = count_v.transform(docs_new)
clf = MultinomialNB()
clf.fit(X_train_counts, train_data.target)

predicted = clf.predict_proba(example_data)
print(predicted)
clf.score(x_target_data, test_data.target)

[[0.00006 0.99994]
 [0.63915 0.36085]]


0.9962264150943396

### Task 3 - Implementujte Naivní Bayesův klasifikátor v situaci, kdy má část příznaků kategorické a část Bernoulliho rozdělení

* První příznak je kategorický (se třemi kategoriemi)
* Zbývajících 10 příznaků má Bernoulliho rozdělení

**Hint** - kategorický příznak převeďte na 3 indikátorové příznaky a odděleně použijte MultinomialNB.
Potom zvlášt odhadněte zbylé Bernoulliho příznaky a na závěr získané pravděpodobnosti pronásobte. 
Pozor - je třeba v jednom z modelů zafixovat rozdělení $Y$ na rovnoměrné - aby se pravděpodobnosti $P(Y = y)$ nenásobili dvakrát.

In [11]:
# Vytvoření datasetu
class_count = 10
X00 = np.random.choice(3, size = (class_count,1), p = [0.4,0.4,0.2])
X01 = np.random.choice(3, size = (class_count,1), p = [0.2,0.5,0.3])
X0 = np.concatenate([X00,X01])
print(X0.shape)

X10 = np.random.choice(2, size = (class_count,5), p = [0.4,0.6])
X11 = np.random.choice(2, size = (class_count,5), p = [0.6,0.4])
X1 = np.concatenate([X10,X11])
print(X1.shape)

X20 = np.random.choice(2, size = (class_count,5), p = [0.4,0.6])
X21 = np.random.choice(2, size = (class_count,5), p = [0.2,0.8])
X2 = np.concatenate([X20,X21])
print(X2.shape)

X = np.concatenate([X0,X1,X2],axis = 1)
print(X.shape)

Y = np.concatenate([np.ones(class_count-3), np.zeros(class_count+3)])
print(Y)

(20, 1)
(20, 5)
(20, 5)
(20, 11)
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [41]:
# Your code here

X0


array([[0],
       [1],
       [2],
       [1],
       [2],
       [0],
       [1],
       [0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [2],
       [2],
       [0],
       [1],
       [1],
       [2]])